## Ideia

Este notebook tem o intuito de pegar os dados disponibilizados neste [google drive](https://drive.google.com/drive/folders/1DrEowqNixud3IlHMO6YTgZqQ2PvgvTpk?usp=sharing) que foi adquirido apos a execução do script neste [github](https://github.com/arthurtorres/Projetos/tree/master/INEP/Criacao_Dados) e transforma-lo num dataset úúnico, cujo a análise seja simploria. Foi escolhido o notebook porque se trata de um dataset de 100MM de linhas, e o computador local que possuo não consegue computar todos os dados.

#### Importando Bibliotecas

In [1]:
import sys
!!{sys.executable} -m pip install dask[dataframe]
!{sys.executable} -m pip install pyarrow
!{sys.executable} -m pip install fastparquet


In [2]:




import dask.dataframe as dd
import pandas as pd

import numpy as np

##Carregando os dados


Os dados estão armazenados no meu google drive, então usarei o seguitne codigo para carregar os dados :

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
folderAluno ="/content/drive/MyDrive/Dados_INEP/Aluno.csv/*"
folderCurso = "/content/drive/MyDrive/Dados_INEP/Curso.csv/*"
folderIES = "/content/drive/MyDrive/Dados_INEP/IES.csv/*"

dataAluno = dd.read_csv(folderAluno,encoding ="ISO-8859-1",dtype={'CO_ALUNO': 'object'})
dataCurso = dd.read_csv(folderCurso,encoding ="ISO-8859-1")
dataIES = dd.read_csv(folderIES,encoding ="ISO-8859-1")


Temos muitos dados repetidos em alunos, pois temos cerca de 100MM de entrada e isto não refleta a população no ensino superior, vamos então remover duplicadas deste dataset

In [5]:
print(f'O Dataset antes da remoção de duplicatas tinha {len(dataAluno)} linhas')
print("Fazendo a Limpeza")
dataAluno = dataAluno.drop_duplicates("CO_ALUNO")
print(f'O dataset apos a remoção de duplicas tem {len(dataAluno)} linhas')
dataAluno = dataAluno.compute()

O Dataset antes da remoção de duplicatas tinha 113190205 linhas
Fazendo a Limpeza
O dataset apos a remoção de duplicas tem 44261224 linhas


# Juntando os datasets e fazendo uma pré limpeza


Vamos olhar o início dos nossos datasets

In [6]:
print(dataAluno.head())
print(dataCurso.head())
print(dataIES.head())

   CO_IES      CO_ALUNO  Cat_Adm  ...  Cota_Racial  Apoio_Social   Ano
0     780  400000004203        4  ...        False         False  2009
1     570  400000004213        1  ...        False         False  2009
2     316  400000004239        4  ...        False         False  2009
3     670  400000004262        6  ...        False         False  2009
4    1131  400000004269        4  ...        False         False  2009

[5 rows x 10 columns]
   CO_CURSO            NO_CURSO
0     10394          ENFERMAGEM
1      5615             DIREITO
2      5618              LETRAS
3     18064            HISTORIA
4      5625  ENGENHARIA QUIMICA
   CO_IES  ... IN_CAPITAL_IES
0     699  ...              1
1    1028  ...              0
2    1189  ...              0
3     275  ...              0
4     580  ...              1

[5 rows x 5 columns]


Vemos que no dataset de Cursos e no dataAluno temos em ambos as variáveis CO_CURSO. Como Curso tem apenas duas colunas, podemos fazer um dicionário com estas colunas, fazer uma associação e colocar a coluna Curso no dataset Aluno. Vamos criar uma função para fazer isto pois pode ser úil futuramente.

In [7]:
#@Criando a função
def dictColuna(datasetvelho,datasetNovo,coluna1,coluna2,nomeNovaColuna) :
  datax = datasetvelho.set_index(coluna1).to_dict()[coluna2]
  a = datasetNovo[coluna1].map(datax)
  datasetNovo[nomeNovaColuna] = a
  

In [8]:
datafinal = dataAluno.copy()

In [9]:
dataCursoLimpo = dataCurso.compute()

Aplicando a função

In [10]:
dictColuna(dataCursoLimpo,datafinal,"CO_CURSO","NO_CURSO","Nome_Curso")



Vendo o resutlado :

In [11]:
datafinal.head()

,CO_IES,CO_ALUNO,Cat_Adm,CO_CURSO,Nivel_Aca,Cor/Raca,Genero,Cota_Racial,Apoio_Social,Ano,Nome_Curso
0,780,400000004203,4,18471,1,2,1,False,False,2009,ARQUITETURA E URBANISMO
1,570,400000004213,1,40171,1,6,0,False,False,2009,ENGENHARIA DA COMPUTAÇÃO
2,316,400000004239,4,82227,1,0,1,False,False,2009,ODONTOLOGIA
3,670,400000004262,6,15474,1,6,0,False,False,2009,ADMINISTRAÇÃO
4,1131,400000004269,4,86830,1,1,1,False,False,2009,ENFERMAGEM


Vemos que foi adicionada uma coluna com o Curso. O Código funcionou bem, faremos parecido com os dados de IES

No dataset referente as IES, temos as variáveis CO_MUNICIPIO_IES e CO_UF_IES que são os códigos do IBGE para cidades e estados. O código sozinho não nos dá uma informação útil, vamos utilizar a função criada anteriormente com uma tabela de correlação, para transformar estes Códigos em Cidades e Estados que dão informação

In [12]:
url = "https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/estados.csv" #estados IBGE
datasEstados = pd.read_csv(url,sep=",",usecols = ["codigo_uf","uf"])
url =  "https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv" #municipios
datasMunicipios = pd.read_csv(url,sep = ",", usecols = ["codigo_ibge","nome"])
datasEstados.columns = ["CO_UF_IES","uf"]
datasMunicipios.columns = ["CO_MUNICIPIO_IES","cidade"]



Utilizando o código para os estados e municipios

In [13]:
dictColuna(datasEstados,dataIES,"CO_UF_IES","uf","UF")
dictColuna(datasMunicipios,dataIES,"CO_MUNICIPIO_IES","cidade","Cidade")


In [14]:
dataIES.head()

,CO_IES,NO_IES,CO_MUNICIPIO_IES,CO_UF_IES,IN_CAPITAL_IES,UF,Cidade
0,699,FUNDACAO UNIVERSIDADE FEDERAL DE RONDONIA,1100205,11,1,RO,Porto Velho
1,1028,UNIVERSIDADE ESTADUAL DE MATO GROSSO DO SUL,5003702,50,0,MS,Dourados
2,1189,UNIVERSIDADE DO PLANALTO CATARINENSE,4209300,42,0,SC,Lages
3,275,UNIVERSIDADE DO VALE DO PARAIBA,3549904,35,0,SP,São José dos Campos
4,580,UNIVERSIDADE FEDERAL DE PERNAMBUCO,2611606,26,1,PE,Recife


Com essas modificações, as colunas CO_UF_IES e CO_MUNICIPIO_IES não se tornam mais úteis, iremos dropá-las.

In [15]:
dataIESClean = dataIES.drop(columns = ["CO_UF_IES","CO_MUNICIPIO_IES"])


Semelhante ao Curso, temos que CO_IES aparece em ambos os datasets, tanto referente aos Institutos, quanto referente aos Alunos. Vamos utilizar nosso código de dicionário para juntar as informações relevantes.


In [16]:
dataIESClean = dataIESClean.compute()

In [17]:
dataIESClean.head()

,CO_IES,NO_IES,IN_CAPITAL_IES,UF,Cidade
0,699,FUNDACAO UNIVERSIDADE FEDERAL DE RONDONIA,1,RO,Porto Velho
1,1028,UNIVERSIDADE ESTADUAL DE MATO GROSSO DO SUL,0,MS,Dourados
2,1189,UNIVERSIDADE DO PLANALTO CATARINENSE,0,SC,Lages
3,275,UNIVERSIDADE DO VALE DO PARAIBA,0,SP,São José dos Campos
4,580,UNIVERSIDADE FEDERAL DE PERNAMBUCO,1,PE,Recife


In [18]:
colunasVelhas = ["NO_IES","Cidade","UF","IN_CAPITAL_IES"]
colunasNovas = ["Nome_Instituição","Municipio","UF","CAPITAL"]
for number in range(len(colunasVelhas)) :
  dictColuna(dataIESClean,datafinal,"CO_IES",colunasVelhas[number],colunasNovas[number])



Vamos olhar o dataset Resultante da junção dos datasets.

In [19]:
datafinal.tail()


,CO_IES,CO_ALUNO,Cat_Adm,CO_CURSO,Nivel_Aca,Cor/Raca,Genero,Cota_Racial,Apoio_Social,Ano,Nome_Curso,Nome_Instituição,Municipio,UF,CAPITAL
30499,24672,1E16F5A689E600ED4D5DCB81941569F3,2,1496238,1,1,2,True,False,2019,ANÁLISE DE PROCESSOS AGROINDUSTRIAIS,FACULDADE DE TECNOLOGIA DE MATÃO,Matão,SP,0
30500,24672,4E9A800BDF87BFB4A8B19E7427418FA6,2,1496238,1,1,2,True,False,2019,ANÁLISE DE PROCESSOS AGROINDUSTRIAIS,FACULDADE DE TECNOLOGIA DE MATÃO,Matão,SP,0
30501,24672,9FD36BFE8F9635ADC07857278DA19DAA,2,1496238,1,3,2,True,False,2019,ANÁLISE DE PROCESSOS AGROINDUSTRIAIS,FACULDADE DE TECNOLOGIA DE MATÃO,Matão,SP,0
30502,24672,D2BFEAC6FFA445580A4417C6FF24F24F,2,1496238,1,1,1,True,False,2019,ANÁLISE DE PROCESSOS AGROINDUSTRIAIS,FACULDADE DE TECNOLOGIA DE MATÃO,Matão,SP,0
30503,24672,F6175BBC0F5B6E6D16012EEA177FDA6B,2,1496238,1,1,2,True,False,2019,ANÁLISE DE PROCESSOS AGROINDUSTRIAIS,FACULDADE DE TECNOLOGIA DE MATÃO,Matão,SP,0


O dataset parece funcionar como o esperado, com todas as informações que queremos para nossa análise, que será feita em outro notebook.


#Últimos Ajustes

Vamos ver os tipos de dados, e ver se podermos melhorar algum para ocupar menos espaço.


In [20]:
datafinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44261224 entries, 0 to 30503
Data columns (total 15 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   CO_IES            int64 
 1   CO_ALUNO          object
 2   Cat_Adm           int64 
 3   CO_CURSO          int64 
 4   Nivel_Aca         int64 
 5   Cor/Raca          int64 
 6   Genero            int64 
 7   Cota_Racial       bool  
 8   Apoio_Social      bool  
 9   Ano               int64 
 10  Nome_Curso        object
 11  Nome_Instituição  object
 12  Municipio         object
 13  UF                object
 14  CAPITAL           int64 
dtypes: bool(2), int64(8), object(5)
memory usage: 4.7+ GB


Podemos retirar as variáveis referentes a códigos, pois temos os seus valores reais. Faremos isto para otimizar os espaços que os dados ocupam na memória.


In [21]:
data = datafinal.drop(columns = ["CO_CURSO","CO_IES","CO_ALUNO"])
                      

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44261224 entries, 0 to 30503
Data columns (total 12 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Cat_Adm           int64 
 1   Nivel_Aca         int64 
 2   Cor/Raca          int64 
 3   Genero            int64 
 4   Cota_Racial       bool  
 5   Apoio_Social      bool  
 6   Ano               int64 
 7   Nome_Curso        object
 8   Nome_Instituição  object
 9   Municipio         object
 10  UF                object
 11  CAPITAL           int64 
dtypes: bool(2), int64(6), object(4)
memory usage: 3.7+ GB


Vemos que este dataset ia ocupar pelo menos 3,7 GB de memoria. Esta quantidade pode ser armazenada na maioria dos agregadores de dados atuais. Eu utilizarei o Google Drive para isto.

Criarei uma pasta com o nome dados_INEP_LIMPO e armazenarei o arquivo em csv, para diminuir ainda o espaço utilizado e facilitar exportações futuras alem de consumir pouca ram.

In [23]:
data.to_csv("/content/drive/MyDrive/dados_INEP_LIMPO/dataClean.csv",chunksize=  100000  ,encoding="ISO-8859-1", index=False)

Por Fim, vamos testar se os dados foram armazenados corretamente.

In [24]:
ds = dd.read_csv("/content/drive/MyDrive/dados_INEP_LIMPO/dataClean.csv",encoding="ISO-8859-1")
print(ds.head())
print(f"O   dataset contem : {len(ds)}  linhas")


   Cat_Adm  Nivel_Aca  Cor/Raca  ...          Municipio  UF  CAPITAL
0        4          1         2  ...             Cuiabá  MT        1
1        1          1         6  ...              Natal  RN        1
2        4          1         0  ...          São Paulo  SP        1
3        6          1         6  ...  Bragança Paulista  SP        0
4        4          1         1  ...           Teresina  PI        1

[5 rows x 12 columns]
O   dataset contem : 44261224  linhas


Após esta preparação de dados, temos um dataset com os Alunos,seus respectivos cursos e suas instituições em um dataset de facil leitura. Vamos trabalhar nele em um notebook futuro. o Link para o notebook para uma análise aprofundada é [este](https://colab.research.google.com/drive/1uD0kRQE90Z8W4g0l4y0Vx_Tif5BN27bp?usp=sharing)
